In [1]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
from datetime import datetime

from dotenv import load_dotenv
import os
import sqlite3 as sql
from utils import login

# Login Process


In [2]:
access_token, fyers = login()
fyers.get_profile()

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=KKSQ1PYF0H-100&redirect_uri=https%3A%2F%2Fsabyabhoi.xyz%2F&response_type=code&state=sample


In [3]:
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'DS05962',
  'name': 'SABYASACHI BHOI',
  'image': None,
  'display_name': 'sabya',
  'pin_change_date': '24-01-2024 10:24:23',
  'email_id': 'sabyabhoi@gmail.com',
  'pwd_change_date': '24-10-2021 14:39:07;90',
  'PAN': '---------',
  'mobile_number': '8527854263',
  'totp': False,
  'pwd_to_expire': 90}}

In [ ]:
fyers_id = os.getenv("FYERS_ID")
redirect_url = os.getenv("REDIRECT_URL")
client_id = os.getenv("CLIENT_ID")
secret_key = os.getenv("SECRET_KEY")
grant_type = "authorization_code"
response_type = "code"
state = "sample"

In [8]:
appSession = fyersModel.SessionModel(
    client_id=client_id,
    redirect_uri=redirect_url,
    response_type=response_type,
    secret_key=secret_key,
    grant_type=grant_type,
    state=state,
)

In [11]:
generateTokenUrl = appSession.generate_authcode()
generateTokenUrl

'https://api-t1.fyers.in/api/v3/generate-authcode?client_id=KKSQ1PYF0H-100&redirect_uri=https%3A%2F%2Fsabyabhoi.xyz%2F&response_type=code&state=sample'

In [13]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDYwODYxMDgsImV4cCI6MTcwNjExNjEwOCwibmJmIjoxNzA2MDg1NTA4LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJEUzA1OTYyIiwib21zIjoiSzEiLCJoc21fa2V5IjoiMzJmNGRiNjBhZWJhZjYyYmU0MTllZDQwNTAzODc1YmFjMDdiYjgzODE1ZWY1NmMzMWU5MzkyMTUiLCJub25jZSI6IiIsImFwcF9pZCI6IktLU1ExUFlGMEgiLCJ1dWlkIjoiYWQ1MTk5ZGU1NDY1NDA0MzlhODlmMWI4MDk1NDQ1ZDQiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.RRP08hs3rcqGVd9-irqQyXTz6f3KzL5v7oGzMtleHk0"

In [14]:
appSession.set_token(auth_code)

response = appSession.generate_token()

In [19]:
try:
    access_token = response["access_token"]
except Exception as e:
    print(e, response)

# Data API


In [16]:
fyers = fyersModel.FyersModel(token=access_token, is_async=False, client_id=client_id)

fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'DS05962',
  'name': 'SABYASACHI BHOI',
  'image': None,
  'display_name': 'sabya',
  'pin_change_date': '24-01-2024 10:24:23',
  'email_id': 'sabyabhoi@gmail.com',
  'pwd_change_date': '24-10-2021 14:39:07;90',
  'PAN': '---------',
  'mobile_number': '8527854263',
  'totp': False,
  'pwd_to_expire': 90}}

In [52]:
db = sql.connect("realtime.db", check_same_thread=False)
c = db.cursor()

s = ["NIFTY50"]


def create_table(tokens):
    for t in tokens:
        print(t)
        c.execute(
            f"CREATE TABLE IF NOT EXISTS {t} (ts datetime primary key, price real(15, 5))"
        )

    try:
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()


create_table(s)

NIFTY24J2517350CE


In [68]:
def onmessage(message):
    """
    Callback function to handle incoming messages from the FyersDataSocket WebSocket.

    Parameters:
        message (dict): The received message from the WebSocket.

    """
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    query = f"INSERT INTO {s[0]} VALUES('{current_time}', {message['ltp']})"
    print(query)

    c.execute(query)

In [69]:
def onerror(message):
    """
    Callback function to handle WebSocket errors.

    Parameters:
        message (dict): The error message received from the WebSocket.


    """
    print("Error:", message)


def onclose(message):
    """
    Callback function to handle WebSocket connection close events.
    """
    print("Connection closed:", message)
    try:
        db.commit()
    except Exception as e:
        print(e)
        db.rollback()


def onopen():
    """
    Callback function to subscribe to data type and symbols upon WebSocket connection.

    """
    # Specify the data type and symbols you want to subscribe to
    data_type = "SymbolUpdate"

    # Subscribe to the specified symbols and data type
    symbols = ["NSE:NIFTY50-INDEX"]
    fyers.subscribe(symbols=symbols, data_type=data_type)
    print(symbols)
    # Keep the socket running to receive real-time data
    fyers.keep_running()

In [70]:
# Create a FyersDataSocket instance with the provided parameters
fyers = data_ws.FyersDataSocket(
    access_token=access_token,  # Access token in the format "appid:accesstoken"
    log_path="",  # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,  # Save response in a log file instead of printing it.
    reconnect=True,  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,  # Callback function to subscribe to data upon connection.
    on_close=onclose,  # Callback function to handle WebSocket connection close events.
    on_error=onerror,  # Callback function to handle WebSocket errors.
    on_message=onmessage,  # Callback function to handle incoming messages from the WebSocket.
)

# Establish a connection to the Fyers WebSocket
fyers.connect()

Error: KeyError: The key 'ltp' is missing in the response.
Error: KeyError: The key 'ltp' is missing in the response.
['NSE:NIFTY50-INDEX']


Error: KeyError: The key 'ltp' is missing in the response.
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:28.966', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:29.379', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:30.380', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:31.404', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:32.380', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:33.395', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:34.389', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:35.423', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:36.391', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:37.385', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:38.387', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:39.388', 21465.9)
INSERT INTO NIFTY24J2517350CE VALUES('2024-01-24 15:31:40.679', 2

In [71]:
fyers.close_connection()